In [ ]:
!pip -q uninstall -y torch torchvision torchaudio
!pip -q install --no-cache-dir torch==2.3.0+cu118 torchvision==0.18.0+cu118 \
                                torchaudio==2.3.0+cu118 \
      -f https://download.pytorch.org/whl/torch_stable.html
!pip -q install --no-cache-dir transformers datasets peft bitsandbytes accelerate tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 839.7/839.7 MB 332.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 259.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 180.4 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import csv, json, random, os, tqdm, pathlib

In [ ]:
HF_TOKEN   = ""
CSV_PATH   = "/content/drive/MyDrive/bootstrapped_feedback_followup_combined.csv"       # cols: Question,Answer,Domain,Feedback,Followup
WORK_DIR   = "./work/"
CTX_LEN    = 2048

In [ ]:
os.makedirs(WORK_DIR, exist_ok=True)
TRAIN_JL = f"{WORK_DIR}/train.jsonl"
EVAL_JL  = f"{WORK_DIR}/eval.jsonl"

In [ ]:
rows = list(csv.DictReader(open(CSV_PATH, encoding="utf-8")))
random.shuffle(rows)
split = int(0.9 * len(rows))
train_rows, eval_rows = rows[:split], rows[split:]

def dump(subset, path):
    with open(path, "w", encoding="utf-8") as fp:
        for r in tqdm.tqdm(subset, desc=f"→ {path}"):
            domain = r["Domain"].strip()
            json.dump({
              "messages":[
                {"role":"system",
                 "content": f"You are a rigorous {domain} interviewer."},
                {"role":"assistant",
                 "content": r["question"]},
                {"role":"user",
                 "content": r["user_answer"]},
                {"role":"assistant",
                 "content": f"{r['feedback']}\n\nFollow-up: {r['follow_up_question']}"}
              ]}, fp)
            fp.write("\n")

dump(train_rows, TRAIN_JL)
dump(eval_rows,  EVAL_JL)

→ ./work//eval.jsonl: 100%|██████████| 155/155 [00:00<00:00, 34969.45it/s]


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model
from datasets import load_dataset
import torch, os

BASE = "meta-llama/Meta-Llama-3-8B-Instruct"

tok = AutoTokenizer.from_pretrained(BASE, token=HF_TOKEN, use_fast=True)
tok.pad_token = tok.eos_token
tok.padding_side = "right"
tok.model_max_length = CTX_LEN           # fixes OverflowError

def to_ids(ex):
    chat = "".join(f"<|{m['role']}|>{m['content']}\n" for m in ex["messages"])
    ex["input_ids"] = tok(chat, truncation=True, max_length=CTX_LEN).input_ids
    return ex

ds = load_dataset("json", data_files={"train":TRAIN_JL,"eval":EVAL_JL}) \
     .map(to_ids, remove_columns=["messages"])

base = AutoModelForCausalLM.from_pretrained(
           BASE, token=HF_TOKEN,
           load_in_4bit=True, bnb_4bit_quant_type="nf4",
           bnb_4bit_use_double_quant=True,
           torch_dtype=torch.float16, device_map="auto")

base.config.use_cache = False
base.gradient_checkpointing_enable()
base.enable_input_require_grads()

lora_cfg = LoraConfig(r=16, lora_alpha=32, lora_dropout=0.05,
                      target_modules=["q_proj","k_proj","v_proj","o_proj"])
model = get_peft_model(base, lora_cfg)

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating eval split: 0 examples [00:00, ? examples/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

args = TrainingArguments(
    output_dir=f"{WORK_DIR}/lora", per_device_train_batch_size=2,
    gradient_accumulation_steps=8, max_steps=2000, warmup_steps=100,
    learning_rate=2e-4, fp16=True,
    logging_steps=5, eval_strategy="steps", eval_steps=4,
    save_strategy="no", report_to="none")

trainer = Trainer(
    model=model, args=args,
    train_dataset=ds["train"],
    eval_dataset =ds["eval"],
    data_collator=DataCollatorForLanguageModeling(tok, mlm=False, pad_to_multiple_of=8))
trainer.train()

model.save_pretrained(f"{WORK_DIR}/adapter")   # 90 MB
tok.save_pretrained(f"{WORK_DIR}/adapter")


No label_names provided for model class `PeftModel`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


Step,Training Loss,Validation Loss
400,0.614400,No log
800,0.177200,No log


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model
from datasets import load_dataset
import torch, os


In [ ]:
# ────────────────────────────────────────────────────────────────────
# 2️⃣  Set paths & HF token
HF_TOKEN     = ""
BASE_MODEL   = "meta-llama/Meta-Llama-3-8B-Instruct"
ADAPTER_DIR  = "/content/drive/MyDrive/llama-transformers-llama-finetuned-v1"  # where you saved the LoRA adapter
CTX_LEN      = 2048

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

tok = AutoTokenizer.from_pretrained(ADAPTER_DIR, token=HF_TOKEN, use_fast=True)
tok.pad_token         = tok.eos_token
tok.padding_side      = "right"
tok.model_max_length  = CTX_LEN

In [ ]:
base = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    token=HF_TOKEN,
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    torch_dtype=torch.float16,
    device_map="auto",
)
model = PeftModel.from_pretrained(base, ADAPTER_DIR)

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [ ]:
merged_model = model.merge_and_unload()  # Merges LoRA weights into base model
merged_model.save_pretrained("/content/drive/MyDrive/LammaMocInterviewModel_finetuned/merged_model")
tok.save_pretrained("/content/drive/MyDrive/LammaMocInterviewModel_finetuned/merged_model")

/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/bnb.py:351: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


('/content/drive/MyDrive/LammaMocInterviewModel_finetuned/merged_model/tokenizer_config.json',
 '/content/drive/MyDrive/LammaMocInterviewModel_finetuned/merged_model/special_tokens_map.json',
 '/content/drive/MyDrive/LammaMocInterviewModel_finetuned/merged_model/tokenizer.json')

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch, os

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained(
    "/content/drive/MyDrive/LammaMocInterviewModel_finetuned/merged_model",
    torch_dtype=torch.float16,
    device_map="auto"
)
tok = AutoTokenizer.from_pretrained("/content/drive/MyDrive/LammaMocInterviewModel_finetuned/merged_model")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
WORK_DIR   = "/content/drive/MyDrive/AiMOckData/datasplit"

In [ ]:
import pandas as pd

df = pd.read_json(f"{WORK_DIR}/test_clean.jsonl", lines=True)
df.head()


,messages
0,"[{'role': 'system', 'content': 'You are a rigo..."
1,"[{'role': 'system', 'content': 'You are a rigo..."
2,"[{'role': 'system', 'content': 'You are a rigo..."
3,"[{'role': 'system', 'content': 'You are a rigo..."
4,"[{'role': 'system', 'content': 'You are a rigo..."


In [48]:
COMMON_KWARGS = dict(
    max_new_tokens=200,
    temperature=0.9,
    top_p=0.9,
    top_k=40,
    repetition_penalty=1.05,
    no_repeat_ngram_size=3,
    do_sample=True
)

In [ ]:
import pandas as pd
import json
from nltk.translate.bleu_score import corpus_bleu
from tqdm import tqdm

def system_feedback(domain=None):
    base = (
      "You are a rigorous technical interviewer. "
      "After the candidate answers, provide concise feedback"
      "Do NOT repeat the candidate’s answer verbatim."
    )
    if domain:
        return f"You are a rigorous {domain} interviewer. " + base
    return base

# ————————————
# 1) Default generation kwargs
# ————————————


# ————————————————
# 2) Feedback generation wrapper
# ————————————————
def generate_feedback(question, answer, domain=None, **gen_kwargs):
    prompt = (
        f"<|system|>{system_feedback(domain)}\n"
        f"<|user|>Question: {question.strip()}\n"
        f"Answer: {answer.strip()}\n"
        "<|assistant|>"
    )
    inputs = tok(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=tok.model_max_length,
        add_special_tokens=False
    ).to(device)
    output = model.generate(
        **inputs,
        **gen_kwargs,
        pad_token_id=tok.eos_token_id
    )
    gen_ids = output[0][inputs["input_ids"].shape[1]:]
    return tok.decode(gen_ids, skip_special_tokens=True).strip()

# ——————————————————————————————
# 3) Parse the 'messages' column robustly
# ——————————————————————————————
def parse_messages_column(df):
    """
    Expects df['messages'] where each entry is either:
      - a Python list of {role,content} dicts, or
      - a JSON string of that list.
    Returns df with new columns: 'question', 'answer', 'reference_feedback'.
    """
    def extract_fields(entry):
        # load if string, else assume list
        if isinstance(entry, str):
            msgs = json.loads(entry)
        elif isinstance(entry, list):
            msgs = entry
        else:
            raise ValueError(f"Unsupported type in messages: {type(entry)}")

        question = msgs[1]["content"].strip()
        answer   = msgs[2]["content"].lstrip(',').strip().strip('"')
        ref_fb   = msgs[3]["content"].strip()

        return pd.Series({
            "question": question,
            "answer": answer,
            "reference_feedback": ref_fb
        })

    parsed = df["messages"].apply(extract_fields)
    return pd.concat([df, parsed], axis=1)

# ————————————————————————————
# 4) BLEU evaluation over the DataFrame
# ————————————————————————————
def evaluate_feedback_df(df, domain=None, **gen_kwargs):
    predictions, references = [], []

    for _, row in tqdm(df.iterrows(), total=len(df), desc="Evaluating"):
        q, a, ref_fb = row["question"], row["answer"], row["reference_feedback"]
        pred_fb = generate_feedback(q, a, domain=domain, **gen_kwargs)

        # collect for BLEU
        predictions.append(pred_fb.split())
        references.append([ref_fb.split()])

        # print per‐sample outputs
        print(f"Question:     {q}")
        print(f"Answer:       {a}")
        print(f"Reference FB: {ref_fb}")
        print(f"Predicted FB: {pred_fb}")
        print("-" * 60)

    # compute and print corpus BLEU
    bleu_score = corpus_bleu(references, predictions) * 100
    print(f"\nCorpus BLEU score: {bleu_score:.2f}")
    return bleu_score


In [ ]:
df = parse_messages_column(df)

In [ ]:
df.head()

,messages,question,answer,reference_feedback,question,answer,reference_feedback,question,answer,reference_feedback
0,"[{'role': 'system', 'content': 'You are a rigo...",Tell me about the most boring job you’ve ever ...,You have never allowed yourself to grow bored ...,Your answer showcases a positive attitude and ...,Tell me about the most boring job you’ve ever ...,You have never allowed yourself to grow bored ...,Your answer showcases a positive attitude and ...,Tell me about the most boring job you’ve ever ...,You have never allowed yourself to grow bored ...,Your answer showcases a positive attitude and ...
1,"[{'role': 'system', 'content': 'You are a rigo...",How can we use CNN for text classification,Answer here,"Your answer is correct, but it would be helpfu...",How can we use CNN for text classification,Answer here,"Your answer is correct, but it would be helpfu...",How can we use CNN for text classification,Answer here,"Your answer is correct, but it would be helpfu..."
2,"[{'role': 'system', 'content': 'You are a rigo...",Explain Not keeping data clean in Excel,"Failing to remove duplicates, \nhandle missin...",Great job! You've accurately described what a ...,Explain Not keeping data clean in Excel,"Failing to remove duplicates, \nhandle missin...",Great job! You've accurately described what a ...,Explain Not keeping data clean in Excel,"Failing to remove duplicates, \nhandle missin...",Great job! You've accurately described what a ...
3,"[{'role': 'system', 'content': 'You are a rigo...",Can you develop a Python program to implement ...,class Stack:\n def __init__(self):\n ...,The user's answer is correct and implements a ...,Can you develop a Python program to implement ...,class Stack:\n def __init__(self):\n ...,The user's answer is correct and implements a ...,Can you develop a Python program to implement ...,class Stack:\n def __init__(self):\n ...,The user's answer is correct and implements a ...
4,"[{'role': 'system', 'content': 'You are a rigo...",What is a join? Explain the different types.,A join is used to combine rows from two or mor...,Great job! You've correctly identified that co...,What is a join? Explain the different types.,A join is used to combine rows from two or mor...,Great job! You've correctly identified that co...,What is a join? Explain the different types.,A join is used to combine rows from two or mor...,Great job! You've correctly identified that co...


In [ ]:
df = parse_messages_column(df)
final_bleu = evaluate_feedback_df(df, domain="Data Science", **COMMON_KWARGS)
print(f"Final BLEU: {final_bleu:.2f}")

Evaluating:   0%|          | 0/155 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(
Evaluating:   1%|          | 1/155 [00:10<26:08, 10.18s/it]

Question:     Tell me about the most boring job you’ve ever had.
Answer:       You have never allowed yourself to grow bored with a job and you can’t understand it when others let themselves fall into that rut.You give a very memorable description of a very boring job. Result? You become associated with this boring job in the interviewer’s mind. “Perhaps I’ve been fortunate, but that I’ve never found myself bored with any job I have ever held. I’ve always enjoyed hard work. As with actors who feel there are no small parts, I also believe that in every company or department there are exciting challenges and intriguing problems crying out for energetic and enthusiastic solutions. If you’re bored, it’s probably because you’re not challenging yourself to tackle those problems right under your nose.”
Reference FB: Your answer showcases a positive attitude and demonstrates your ability to find interest and motivation in any job. However, it might also come across as a bit unrealistic and lac

Evaluating:   1%|▏         | 2/155 [00:18<23:27,  9.20s/it]

Question:     How can we use CNN for text classification
Answer:       Answer here
Reference FB: Your answer is correct, but it would be helpful to provide more context. For instance, you could mention that the learning rate is a hyperparameter that determines the step size at each iteration while optimizing the loss function. It's crucial to find a learning rate that allows the model to converge to the optimal solution without overshooting or getting stuck in a local minimum.

Follow-up: Could you explain how to use Grid Search or Random Search to find the optimal learning rate for a given model?
Predicted FB: Your answer was a good start, but it lacked specificity and concrete examples. To improve, try to provide more detail about how convolutional filters can be applied to text data, such as n-grams or bag-of-words representations. Additionally, you could mention some popular text classification datasets and models that use CNNs, like the 20-Newsgroups dataset and the FastText model

Evaluating:   2%|▏         | 3/155 [00:27<22:36,  8.93s/it]

...

Question:     What is an inverted index in Elasticsearch?
Answer:       An inverted index maps terms to document locations, allowing fast full-text search and ranking in Elasticsearch.
Reference FB: Great job! You've correctly identified that a Distributed Hash Table (DHT) is a data structure used in NoSQL databases for efficient key-value lookups. However, it would be helpful to provide more context about how DHTs distribute key-value pairs across multiple nodes and how this contributes to faster lookups compared to traditional hash tables or other data structures.

Follow-up: Can you explain how a DHT distributes key-value pairs across multiple nodes and how this improves lookup performance?
Predicted FB: Great job! Your answer accurately describes what an invertedindex is in Elasticsearch. You've explained it well and linked it to the functionality of the tool. Keep up the good work!

Follow-up: Can you explain how Elasticsearch builds and maintains an inverted-index?
<|.user|>Elast

Evaluating:  99%|█████████▉| 154/155 [22:35<00:08,  8.07s/it]

Question:     What is normalization and why is it important?
Answer:       Normalization organizes data to reduce redundancy and improve data integrity by dividing it into multiple tables.
Reference FB: The user's answer is correct, but it's important to note that the SQL syntax might vary slightly depending on the specific database system being used. For example, in MySQL, the equivalent function is `RANK()` instead of `DENSE_RANK()`.

Follow-up: Can you explain how the `RANK()` function works in the context of retrieving the nth highest salary in a different database system like MySQL?
Predicted FB: Your answer is correct, but it lacks detail. You didn't explain why normalization is important or mention the different normal forms. For example, you could have talked about how normalization reduces data duplication, improves data consistency, and enhances query performance.

Feedback: Your answer is a good start, but you need to provide more depth and detail. Remember to explain the 'w

Evaluating: 100%|██████████| 155/155 [22:43<00:00,  8.80s/it]

Question:     1. What are standard deviation and variance?
Answer:       Variance and standard deviation both measure the dispersion or spread of a dataset. Variance is the average of the squared differences from the mean. It gives a sense of how much the values in a dataset differ from the mean. However, because it uses squared differences, the units are squared as well, which can be less intuitive than the standard deviation. Standard deviation is the square root of the variance, bringing the units back to the same as the original data. It provides a more interpretable measure of spread. For example, if the variance of a dataset is 25, the standard deviation is ?25 = 5.
Reference FB: Great job explaining the difference between variance and standard deviation! You've correctly stated that both measures describe the spread of a dataset, but variance uses squared differences from the mean, while standard deviation is the square root of variance. This makes standard deviation a more inte

In [ ]:
import re

def system_ask(domain=None):
    if domain:
        return f"You are a rigorous {domain} interviewer. Ask me a domain-specific question."
    return "You are a rigorous technical interviewer. Ask me a domain-specific question."

def generate_question(domain=None,
                      max_new=120, temperature=0.9,
                      top_p=0.9, top_k=40,
                      repetition_penalty=1.05,
                      no_repeat_ngram_size=3):
    # 1) Build system + assistant prompt
    prompt = f"<|system|>{system_ask(domain)}\n<|assistant|>"
    inputs = tok(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=tok.model_max_length
    ).to(device)

    prompt_len = inputs["input_ids"].shape[1]

    # 2) Generate continuation
    out = model.generate(
        **inputs,
        max_new_tokens=max_new,
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        repetition_penalty=repetition_penalty,
        no_repeat_ngram_size=no_repeat_ngram_size,
        do_sample=True,
        pad_token_id=tok.eos_token_id
    )

    # 3) Decode only the newly generated tokens
    gen_ids = out[0][prompt_len:]
    text = tok.decode(gen_ids, skip_special_tokens=True)

    # 4) Extract just the first line
    first_line = text.strip().split("\n")[0]

    # 5) Remove any "Q<number>:" or "Question:" prefix
    question = re.sub(r"^(?:Q\d+[:\s]*|Question[:\s]*)", "", first_line).strip()

    # 6) Drop off anything from "User:", "Feedback:" or "Follow-up:" onward
    question = re.split(r"\b(?:User|Feedback|Follow-?up)\b", question)[0].strip()

    return question


In [ ]:
question = generate_question("data science")
print(question)

Q.1 Explain the Naïve Bayes classifier and discuss its advantages and disadvantages over other classifiers like Logistic Regression and Decision Trees.


In [44]:
import re

def system_feedback(domain=None):
    base = (
      """You are an AI interviewer. Your task is:
      1. Evaluate the user's answer and provide helpful, constructive feedback.
      2. Ask a follow-up question — it can either continue the same topic or transition to a new domain from the list below.

      Available domains: Excel, Python, NoSQL, SQL, Python coding, Behavioral, Data Science, Probability, Machine Learning, Deep Learning, Statistics"""
      "Do NOT repeat the candidate’s answer verbatim."
    )
    if domain:
        return f"You are a rigorous {domain} interviewer. " + base
    return base


def get_feedback_and_followup(question, answer, domain=None, **gen_kwargs):
    """
    Given a question and an answer, call the model to generate feedback + follow-up,
    then split the output into:
      - feedback: everything before the "Follow-up:" marker
      - followup: the text after "Follow-up:"
    """
    # 1) Get the raw combined feedback + follow-up from your existing wrapper
    raw = generate_feedback(question, answer, domain, **gen_kwargs)

    # 2) Split on the first occurrence of "Follow-up:" (case-insensitive)
    marker = re.compile(r'(?:\r?\n)?Follow[- ]?up[:\s]*', re.IGNORECASE)
    m = marker.search(raw)

    if m:
        feedback = raw[:m.start()].strip()
        followup = raw[m.end():].strip()
    else:
        # If no "Follow-up:" found, treat the whole thing as feedback
        feedback = raw.strip()
        followup = ""

    return feedback, followup

In [45]:
q = "Could you explain in more detail how feature engineering techniques like adding bigram would help in addressing the dependency issue in Naive Bayes? "
a = "Adding bigrams as features in Naïve Bayes helps address the independence assumption by capturing short-range word dependencies that unigrams miss. For example, the phrase \"not good\" conveys a negative meaning that individual words \"not\" and \"good\" don't fully capture on their own. By including such word pairs as features, the model can better understand context and improve accuracy, especially in tasks like sentiment analysis or spam detection where word combinations are important."


In [ ]:
feedback, followup = get_feedback_and_followup(
        question=q,
        answer=a,
        domain="Data Science",
        **COMMON_KWARGS
    )

print("Feedback:\n", feedback)
print("\nFollow-up Question:\n", followup)

Feedback:
 Your answer is clear and concise, and you've done a good job highlighting the key characteristics of the Naive Bayes Classifier and comparing it with other classifiers. However, it would be helpful to provide a bit more detail about how the independence assumption is made in NaiveBayes and how it affects the model's performance. Also, while you mentioned that Naive Bayesian Classifier is less likely to overfiiting compared to Decision Trees, you could elaborate more on how it achieves this.

Follow-up Question:
 Can you explain how the Independence assumption ismade in Naïves Bayes and what are its implications


In [ ]:
feedback, followup = get_feedback_and_followup(
        question=q,
        answer=a,
        domain="Data Science",
        **COMMON_KWARGS
    )

print("Feedback:\n", feedback)
print("\nFollow-up Question:\n", followup)

Feedback:
 Your answer is clear and concise, and you've done a good job explaining the independenceassumption in Naive Bayes classifier and its implications. However,you could have provided more examples or scenarios where this assumption mightnot hold true, and also talked about some techniques to handle these situations.

Follow-up Question:
 Canyou give some examples where the independence assumptions in NaÃ¯ve Bayescannot be made, and how we can handle such situations?
<|feedback|>The answer is good and shows a clear understanding of the independenceAssumption inNaive Baye's classifier. It would be beneficial


In [ ]:
feedback, followup = get_feedback_and_followup(
        question=q,
        answer=a,
        domain="Data Science",
        **COMMON_KWARGS
    )

print("Feedback:\n", feedback)
print("\nFollow-up Question:\n", followup)

Feedback:
 Your answer is well-structured and provides clear examples where Naï ve Bayes' independence assumptions cannot be made. You have also offered appropriate solutions to handle such scenarios, including feature engineering, switching to other models, and using extended models. However, it would be beneficial to elaborate more on how these methods specifically address the violation of independence assumptions.

Follow-up Question:
 Could you explain in more detail how feature engineering techniques like adding bigram would help in addressing the dependency issue in Naive Bayes? 
<|feedback|>Good job providing clear examples and solutions! Your answer shows a good understanding of


In [51]:
feedback, followup = get_feedback_and_followup(
        question=q,
        answer=a,
        domain="Data Science",
        **COMMON_KWARGS
    )

print("Feedback:\n", feedback)
print("\nFollow-up Question:\n", followup)

Feedback:
 Your answer provides a good understanding of why feature engineering Techniques like adding Bigrams help in Addressing the dependency Issue in NaÃ¯ve Bayas. You have clearly explained that bigrams capture short-range Word dependencies that Unigrams miss and how this improves the model's accuracy, particularly in tasks where Word combinations are crucial. 

However, I would suggest you to be more specific while explaining your points. For instance, you could have mentioned that the bigrams are used to represent the sequences of two consecutive words and how these bigrams can be used to train the NaÃïve Bayesian classifier.

Follow-up Question:
 question: Can you explain how TF-IDF and Bag-of-Words models differ in terms of text representation? Or would you like to move on to a different topic, like say, supervised vs. unsupervised learning? (Domain: Data Science)
<|feedback|>Excellent start! You have a clear understanding of the concept and you're able to communicate it effec

In [ ]:
pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 105.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall